In [1]:
#Imported the needed libraries for analysis
import warnings 
warnings.filterwarnings ('ignore')

import seaborn as sns
import re
import os 
import gzip 
from collections import Counter 
from pprint import pprint

import pandas as pd 
import numpy as np
import tweepy
import csv

import matplotlib 
import matplotlib.pyplot as plt

np.random.seed(123)

import string 
import nltk
from nltk.corpus import stopwords 
from nltk.text import TextCollection 
from nltk.collocations import BigramCollocationFinder 
from nltk.metrics.association import BigramAssocMeasures
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords 
from nltk.text import TextCollection 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob


import sklearn 
from sklearn.manifold import TSNE 
from sklearn.metrics.pairwise import cosine_similarity

from tqdm.notebook import tqdm

import watermark

%load_ext watermark

%matplotlib inline

# Data Gathering

In this section, we focused on using tweepy to scrape tweeets from twitter.
10,000 tweets per candidate on the 18th of November, 2022.


In [2]:
consumer_key = '3m6LjclW3VpMiF8BfbYW4gDqA'
consumer_secret = 'x8sWJU444fPDabqenAwhBwWIaQRKPG3S1gqvN54GbwMXCmU5zA'
access_token = '383866844-HwLH0lUbulN1oNb5pz8IeBV0ahSiJXu7CiNVpnBj'
access_secret = 'N6dv2OVdwLZhnyPiNHZu10VrCyyCrEbgBXuDYF94yaPGM'


In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)


In [ ]:
# assign header columns
header = ['Time', 'Text']

In [4]:
# Open/Create a file to append data
csvFile = open('Peter_Obi.csv', 'a')

# open CSV file and assign header
with open("Peter_Obi.csv", 'w') as file:
    dw = csv.DictWriter(file, delimiter=',', 
                        fieldnames=header)
    dw.writeheader()
  

In [6]:
#Use csv Writer
csvWriter = csv.writer(csvFile)

In [7]:
tweets=[]
for tweet in tweepy.Cursor(api.search_tweets,q="Peter Obi",
                           lang="en",
                           since_id="2022-11-18").items(10000):
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])
    tweets.append(tweet)

print("Total tweets fetched:", len(tweets))


Rate limit reached. Sleeping for: 337
Rate limit reached. Sleeping for: 833
Rate limit reached. Sleeping for: 843
Rate limit reached. Sleeping for: 840


Total tweets fetched: 10000


In [8]:
# Open/Create a file to append data
csvFile = open('Asiwaju_Tinubu.csv', 'a')

# open CSV file and assign header
with open("Asiwaju_Tinubu.csv", 'w') as file:
    dw = csv.DictWriter(file, delimiter=',', 
                        fieldnames=header)
    dw.writeheader()
  

In [9]:
#Use csv Writer
csvWriter = csv.writer(csvFile)

In [10]:
tweets=[]
for tweet in tweepy.Cursor(api.search_tweets,q="Bola Ahmed Tinubu",
                           lang="en",
                           since_id="2022-11-18").items(10000):
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])
    tweets.append(tweet)

print("Total tweets fetched:", len(tweets))


Rate limit reached. Sleeping for: 717
Rate limit reached. Sleeping for: 845
Rate limit reached. Sleeping for: 844


Total tweets fetched: 10000


In [11]:
# Open/Create a file to append data
csvFile = open('Atiku_Abubakar.csv', 'a')

# open CSV file and assign header
with open("Atiku_Abubakar.csv", 'w') as file:
    dw = csv.DictWriter(file, delimiter=',', 
                        fieldnames=header)
    dw.writeheader()
  

In [12]:
#Use csv Writer
csvWriter = csv.writer(csvFile)

In [13]:
tweets=[]
for tweet in tweepy.Cursor(api.search_tweets,q="Atiku Abubakar",
                           lang="en",
                           since_id="2022-11-18").items(10000):
    csvWriter.writerow([tweet.created_at, tweet.text.encode('utf-8')])
    tweets.append(tweet)

print("Total tweets fetched:", len(tweets))


Rate limit reached. Sleeping for: 420
Rate limit reached. Sleeping for: 846
Rate limit reached. Sleeping for: 846
Rate limit reached. Sleeping for: 846


Total tweets fetched: 10000


# Data Cleaning

In [2]:
peter_df=pd.read_csv("Peter_Obi.csv")

In [3]:
peter_df['ID'] = range(1, len(peter_df) + 1)
#peter_df.set_index('ID', inplace=True)

In [4]:
peter_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Time    10000 non-null  object
 1   Text    10000 non-null  object
 2   ID      10000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 234.5+ KB


In [5]:
peter_df.head()

,Time,Text,ID
0,2022-11-18 22:50:30+00:00,b'RT @Sports_Doctor2: Peter Obi as Governor of...,1
1,2022-11-18 22:50:26+00:00,b'RT @jeffphilips1: Peter Obi is planning to c...,2
2,2022-11-18 22:50:20+00:00,b'RT @NnamdiChife: Peter Obi will win the FCT.',3
3,2022-11-18 22:50:18+00:00,b'RT @BishopPOEvang: We had a blast time today...,4
4,2022-11-18 22:50:18+00:00,"b""RT @AishaYesufu: When Peter Obi @PeterObi sp...",5


In [6]:
tinubu_df=pd.read_csv("Asiwaju_Tinubu.csv")

In [7]:
tinubu_df['ID'] = range(1, len(tinubu_df) + 1)
#tinubu_df.set_index('ID', inplace=True)

In [8]:
tinubu_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Time    10000 non-null  object
 1   Text    10000 non-null  object
 2   ID      10000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 234.5+ KB


In [9]:
tinubu_df.head()

,Time,Text,ID
0,2022-11-18 23:44:03+00:00,"b""RT @DavidHundeyin: For those who still haven...",1
1,2022-11-18 23:43:44+00:00,"b""RT @DavidHundeyin: For those who still haven...",2
2,2022-11-18 23:43:27+00:00,"b""RT @DavidHundeyin: For those who still haven...",3
3,2022-11-18 23:43:11+00:00,"b""RT @DavidHundeyin: For those who still haven...",4
4,2022-11-18 23:43:02+00:00,b'RT @JeffreyGuterman: Tinubu submitted forged...,5


In [10]:
atiku_df=pd.read_csv("Atiku_Abubakar.csv")

In [11]:
atiku_df['ID'] = range(1, len(atiku_df) + 1)
#atiku_df.set_index('ID', inplace=True)

In [12]:
atiku_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Time    10000 non-null  object
 1   Text    10000 non-null  object
 2   ID      10000 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 234.5+ KB


In [13]:
atiku_df.head()

,Time,Text,ID
0,2022-11-19 00:33:01+00:00,"b""RT @Ucijomanta: Restructuring,\nRule of Law,...",1
1,2022-11-19 00:26:21+00:00,b'RT @Capt_Aliyu: Buhari will go down as the w...,2
2,2022-11-19 00:18:57+00:00,b'RT @Dec_Uche: Atiku Abubakar is 360 degrees ...,3
3,2022-11-19 00:16:53+00:00,b'RT @Tony_Ogbuagu: If there is anything you w...,4
4,2022-11-19 00:16:24+00:00,b'RT @cousinray_: BREAKING NEWS!! \nMFN Coalit...,5


In [14]:
clean_tweets=peter_df['Text']
peter_df['Clean_Tweets']=clean_tweets

In [15]:
#convert the text in the Text column to lowercase

peter_df['Clean_Tweets'] = peter_df['Clean_Tweets'].str.lower()

#remove usernames
peter_df['Clean_Tweets'] = peter_df['Clean_Tweets'].str.replace('@[\w]+','', regex=True)


peter_df['Clean_Tweets'] = peter_df['Clean_Tweets'].str.replace('b\'rt\s','', regex=True)

peter_df['Clean_Tweets'] = peter_df['Clean_Tweets'].str.replace('b\"rt\s','', regex=True)

peter_df['Clean_Tweets'] = peter_df['Clean_Tweets'].str.replace('\n','', regex=True)

peter_df['Clean_Tweets'] = peter_df['Clean_Tweets'].str.replace('\n','', regex=True)

#remove non-alphanumeric characters
peter_df['Clean_Tweets'] = peter_df['Clean_Tweets'].str.replace('[^a-z0-9]',' ', regex=True)

#remove links
peter_df['Clean_Tweets'] = peter_df['Clean_Tweets'].str.replace('http\S+','', regex=True)

peter_df['Clean_Tweets'] = peter_df['Clean_Tweets'].str.replace('www.\S+','', regex=True)

#remove hashtags
peter_df['Clean_Tweets'] = peter_df['Clean_Tweets'].str.replace('#[A-Za-z0-9_]','', regex=True)

#remove multiple spaces
peter_df['Clean_Tweets'] = peter_df['Clean_Tweets'].str.replace('\s+',' ', regex=True)

#remove single characters
peter_df['Clean_Tweets'] = peter_df['Clean_Tweets'].str.replace('\s+[a-zA-Z]\s+',' ', regex=True)

#remove digits
peter_df['Clean_Tweets'] = peter_df['Clean_Tweets'].str.replace('\d+','', regex=True)

In [16]:
peter_df.head()

,Time,Text,ID,Clean_Tweets
0,2022-11-18 22:50:30+00:00,b'RT @Sports_Doctor2: Peter Obi as Governor of...,1,peter obi as governor of anambra state invite...
1,2022-11-18 22:50:26+00:00,b'RT @jeffphilips1: Peter Obi is planning to c...,2,peter obi is planning to come and render all ...
2,2022-11-18 22:50:20+00:00,b'RT @NnamdiChife: Peter Obi will win the FCT.',3,peter obi will win the fct
3,2022-11-18 22:50:18+00:00,b'RT @BishopPOEvang: We had a blast time today...,4,we had blast time today sensitization for gra...
4,2022-11-18 22:50:18+00:00,"b""RT @AishaYesufu: When Peter Obi @PeterObi sp...",5,when peter obi speaks don hold my breath know...


In [17]:
clean_tweets=tinubu_df['Text']
tinubu_df['Clean_Tweets']=clean_tweets

In [18]:
#convert the text in the Text column to lowercase

tinubu_df['Clean_Tweets'] = tinubu_df['Clean_Tweets'].str.lower()

#remove usernames
tinubu_df['Clean_Tweets'] = tinubu_df['Clean_Tweets'].str.replace('@[\w]+','', regex=True)


tinubu_df['Clean_Tweets'] = tinubu_df['Clean_Tweets'].str.replace('b\'rt\s','', regex=True)

tinubu_df['Clean_Tweets'] = tinubu_df['Clean_Tweets'].str.replace('b\"rt\s','', regex=True)

tinubu_df['Clean_Tweets'] = tinubu_df['Clean_Tweets'].str.replace('\n','', regex=True)

tinubu_df['Clean_Tweets'] = tinubu_df['Clean_Tweets'].str.replace('\n','', regex=True)


#remove non-alphanumeric characters
tinubu_df['Clean_Tweets'] = tinubu_df['Clean_Tweets'].str.replace('[^a-z0-9]',' ', regex=True)

#remove links
tinubu_df['Clean_Tweets'] = tinubu_df['Clean_Tweets'].str.replace('http\S+','', regex=True)

tinubu_df['Clean_Tweets'] = tinubu_df['Clean_Tweets'].str.replace('www.\S+','', regex=True)


#remove hashtags
tinubu_df['Clean_Tweets'] = tinubu_df['Clean_Tweets'].str.replace('#[A-Za-z0-9_]','', regex=True)

#remove multiple spaces
tinubu_df['Clean_Tweets'] = tinubu_df['Clean_Tweets'].str.replace('\s+',' ', regex=True)

#remove single characters
tinubu_df['Clean_Tweets'] = tinubu_df['Clean_Tweets'].str.replace('\s+[a-zA-Z]\s+',' ', regex=True)

#remove digits
tinubu_df['Clean_Tweets'] = tinubu_df['Clean_Tweets'].str.replace('\d+','', regex=True)

In [19]:
tinubu_df.head()

,Time,Text,ID,Clean_Tweets
0,2022-11-18 23:44:03+00:00,"b""RT @DavidHundeyin: For those who still haven...",1,for those who still haven read it this is the...
1,2022-11-18 23:43:44+00:00,"b""RT @DavidHundeyin: For those who still haven...",2,for those who still haven read it this is the...
2,2022-11-18 23:43:27+00:00,"b""RT @DavidHundeyin: For those who still haven...",3,for those who still haven read it this is the...
3,2022-11-18 23:43:11+00:00,"b""RT @DavidHundeyin: For those who still haven...",4,for those who still haven read it this is the...
4,2022-11-18 23:43:02+00:00,b'RT @JeffreyGuterman: Tinubu submitted forged...,5,tinubu submitted forged documents and committ...


In [20]:
clean_tweets=atiku_df['Text']
atiku_df['Clean_Tweets']=clean_tweets

In [21]:
#convert the text in the Text column to lowercase

atiku_df['Clean_Tweets'] = atiku_df['Clean_Tweets'].str.lower()

#remove usernames
atiku_df['Clean_Tweets'] = atiku_df['Clean_Tweets'].str.replace('@[\w]+','', regex=True)


atiku_df['Clean_Tweets'] = atiku_df['Clean_Tweets'].str.replace('b\'rt\s','', regex=True)

atiku_df['Clean_Tweets'] = atiku_df['Clean_Tweets'].str.replace('b\"rt\s','', regex=True)

atiku_df['Clean_Tweets'] = atiku_df['Clean_Tweets'].str.replace('\n','', regex=True)

atiku_df['Clean_Tweets'] = atiku_df['Clean_Tweets'].str.replace('\t','', regex=True)

#remove non-alphanumeric characters
atiku_df['Clean_Tweets'] = atiku_df['Clean_Tweets'].str.replace('[^a-z0-9]',' ', regex=True)

#remove links
atiku_df['Clean_Tweets'] = atiku_df['Clean_Tweets'].str.replace('http\S+','', regex=True)

atiku_df['Clean_Tweets'] = atiku_df['Clean_Tweets'].str.replace('www.\S+','', regex=True)

#remove hashtags
atiku_df['Clean_Tweets'] = atiku_df['Clean_Tweets'].str.replace('#[A-Za-z0-9_]','', regex=True)

#remove multiple spaces
atiku_df['Clean_Tweets'] = atiku_df['Clean_Tweets'].str.replace('\s+',' ', regex=True)

#remove single characters
atiku_df['Clean_Tweets'] = atiku_df['Clean_Tweets'].str.replace('\s+[a-zA-Z]\s+',' ', regex=True)

#remove digits
atiku_df['Clean_Tweets'] = atiku_df['Clean_Tweets'].str.replace('\d+','', regex=True)

In [22]:
atiku_df.head()

,Time,Text,ID,Clean_Tweets
0,2022-11-19 00:33:01+00:00,"b""RT @Ucijomanta: Restructuring,\nRule of Law,...",1,restructuring nrule of law ntrue federalism n...
1,2022-11-19 00:26:21+00:00,b'RT @Capt_Aliyu: Buhari will go down as the w...,2,buhari will go down as the worse president in...
2,2022-11-19 00:18:57+00:00,b'RT @Dec_Uche: Atiku Abubakar is 360 degrees ...,3,atiku abubakar is degrees ahead of other pre...
3,2022-11-19 00:16:53+00:00,b'RT @Tony_Ogbuagu: If there is anything you w...,4,if there is anything you will miss don xe x x...
4,2022-11-19 00:16:24+00:00,b'RT @cousinray_: BREAKING NEWS!! \nMFN Coalit...,5,breaking news nmfn coalition condemns atiku a...


In [23]:
#tokenize the text in Clean_Tweets columnn and store in a new column called tweet_tokens
def apwords(words):
    filtered_sentence = []
    words = word_tokenize(words)
    for w in words:
        filtered_sentence.append(w)
    return filtered_sentence
addwords = lambda x: apwords(x)

In [24]:
peter_df['Tweet_tokens'] = peter_df['Clean_Tweets'].apply(addwords)

In [25]:
tinubu_df['Tweet_tokens'] = tinubu_df['Clean_Tweets'].apply(addwords)

In [26]:
atiku_df['Tweet_tokens'] = atiku_df['Clean_Tweets'].apply(addwords)

In [27]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [28]:
#remove stopwords from token_list and store in a new column called 'no_stop_tokens'

peter_df['no_stop_tokens']= peter_df['Tweet_tokens'].apply(lambda x: [item for item in x if item not in stop])
tinubu_df['no_stop_tokens']= tinubu_df['Tweet_tokens'].apply(lambda x: [item for item in x if item not in stop])
atiku_df['no_stop_tokens']= atiku_df['Tweet_tokens'].apply(lambda x: [item for item in x if item not in stop])


In [29]:
#calling the SentimentIntensityAnalyzer into sentiment

sentiment = SentimentIntensityAnalyzer()

In [30]:
#Using sentiment process the no_stop_tokens column to get the neg, neu, pos and compound scores

peter_result = {}
for i, row in tqdm(peter_df.iterrows(),total=len(peter_df)):
    text=row['Clean_Tweets']
    myid=row['ID']
    peter_result[myid]=sentiment.polarity_scores(text)
    

  0%|          | 0/10000 [00:00<?, ?it/s]

In [31]:
#append the results from sentiment analysis to peter_df

peter_result_df=pd.DataFrame(peter_result).T
peter_result_df=peter_result_df.reset_index().rename(columns={'index':'ID'})
peter_df=peter_df.merge(peter_result_df)

In [32]:
#Using sentiment process the no_stop_tokens column to get the neg, neu, pos and compound scores

tinubu_result = {}
for i, row in tqdm(tinubu_df.iterrows(),total=len(tinubu_df)):
    text=row['Clean_Tweets']
    myid=row['ID']
    tinubu_result[myid]=sentiment.polarity_scores(text)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [33]:
#append the results from sentiment analysis to tinubu_df

tinubu_result_df=pd.DataFrame(tinubu_result).T
tinubu_result_df=tinubu_result_df.reset_index().rename(columns={'index':'ID'})
tinubu_df=tinubu_df.merge(tinubu_result_df)

In [34]:
#Using sentiment process the no_stop_tokens column to get the neg, neu, pos and compound scores

atiku_result = {}
for i, row in tqdm(atiku_df.iterrows(),total=len(atiku_df)):
    text=row['Clean_Tweets']
    myid=row['ID']
    atiku_result[myid]=sentiment.polarity_scores(text)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [35]:
#append the results from sentiment analysis to atiku_df

atiku_result_df=pd.DataFrame(atiku_result).T
atiku_result_df=atiku_result_df.reset_index().rename(columns={'index':'ID'})
atiku_df=atiku_df.merge(atiku_result_df)

In [36]:
# specify positive and negative compound scores for each review

peter_df.loc[peter_df['compound'] > 0, 'comp_score'] = 'positive'
peter_df.loc[peter_df['compound'] < 0, 'comp_score'] = 'negative'
peter_df.loc[peter_df['compound'] == 0, 'comp_score'] = 'neutral'

tinubu_df.loc[tinubu_df['compound'] > 0, 'comp_score'] = 'positive'
tinubu_df.loc[tinubu_df['compound'] < 0, 'comp_score'] = 'negative'
tinubu_df.loc[tinubu_df['compound'] == 0, 'comp_score'] = 'neutral'

atiku_df.loc[atiku_df['compound'] > 0, 'comp_score'] = 'positive'
atiku_df.loc[atiku_df['compound'] < 0, 'comp_score'] = 'negative'
atiku_df.loc[atiku_df['compound'] == 0, 'comp_score'] = 'neutral'


In [37]:
peter_df.columns

Index(['Time', 'Text', 'ID', 'Clean_Tweets', 'Tweet_tokens', 'no_stop_tokens',
       'neg', 'neu', 'pos', 'compound', 'comp_score'],
      dtype='object')

In [38]:
peter_df.head()

,Time,Text,ID,Clean_Tweets,Tweet_tokens,no_stop_tokens,neg,neu,pos,compound,comp_score
0,2022-11-18 22:50:30+00:00,b'RT @Sports_Doctor2: Peter Obi as Governor of...,1,peter obi as governor of anambra state invite...,"[peter, obi, as, governor, of, anambra, state,...","[peter, obi, governor, anambra, state, invited...",0.000,1.000,0.000,0.0000,neutral
1,2022-11-18 22:50:26+00:00,b'RT @jeffphilips1: Peter Obi is planning to c...,2,peter obi is planning to come and render all ...,"[peter, obi, is, planning, to, come, and, rend...","[peter, obi, planning, come, render, investmen...",0.123,0.877,0.000,-0.4215,negative
2,2022-11-18 22:50:20+00:00,b'RT @NnamdiChife: Peter Obi will win the FCT.',3,peter obi will win the fct,"[peter, obi, will, win, the, fct]","[peter, obi, win, fct]",0.000,0.568,0.432,0.5859,positive
3,2022-11-18 22:50:18+00:00,b'RT @BishopPOEvang: We had a blast time today...,4,we had blast time today sensitization for gra...,"[we, had, blast, time, today, sensitization, f...","[blast, time, today, sensitization, grassroot,...",0.000,1.000,0.000,0.0000,neutral
4,2022-11-18 22:50:18+00:00,"b""RT @AishaYesufu: When Peter Obi @PeterObi sp...",5,when peter obi speaks don hold my breath know...,"[when, peter, obi, speaks, don, hold, my, brea...","[peter, obi, speaks, hold, breath, know, capab...",0.000,0.894,0.106,0.3818,positive


In [39]:
peter_df.comp_score.value_counts()

positive    4105
negative    3065
neutral     2830
Name: comp_score, dtype: int64

In [40]:
tinubu_df.head()

,Time,Text,ID,Clean_Tweets,Tweet_tokens,no_stop_tokens,neg,neu,pos,compound,comp_score
0,2022-11-18 23:44:03+00:00,"b""RT @DavidHundeyin: For those who still haven...",1,for those who still haven read it this is the...,"[for, those, who, still, haven, read, it, this...","[still, read, story, contains, receipts, showi...",0.000,1.000,0.000,0.0000,neutral
1,2022-11-18 23:43:44+00:00,"b""RT @DavidHundeyin: For those who still haven...",2,for those who still haven read it this is the...,"[for, those, who, still, haven, read, it, this...","[still, read, story, contains, receipts, showi...",0.000,1.000,0.000,0.0000,neutral
2,2022-11-18 23:43:27+00:00,"b""RT @DavidHundeyin: For those who still haven...",3,for those who still haven read it this is the...,"[for, those, who, still, haven, read, it, this...","[still, read, story, contains, receipts, showi...",0.000,1.000,0.000,0.0000,neutral
3,2022-11-18 23:43:11+00:00,"b""RT @DavidHundeyin: For those who still haven...",4,for those who still haven read it this is the...,"[for, those, who, still, haven, read, it, this...","[still, read, story, contains, receipts, showi...",0.000,1.000,0.000,0.0000,neutral
4,2022-11-18 23:43:02+00:00,b'RT @JeffreyGuterman: Tinubu submitted forged...,5,tinubu submitted forged documents and committ...,"[tinubu, submitted, forged, documents, and, co...","[tinubu, submitted, forged, documents, committ...",0.181,0.688,0.131,-0.2023,negative


In [41]:
tinubu_df.comp_score.value_counts()

neutral     5176
positive    4143
negative     681
Name: comp_score, dtype: int64

In [42]:
atiku_df.head()

,Time,Text,ID,Clean_Tweets,Tweet_tokens,no_stop_tokens,neg,neu,pos,compound,comp_score
0,2022-11-19 00:33:01+00:00,"b""RT @Ucijomanta: Restructuring,\nRule of Law,...",1,restructuring nrule of law ntrue federalism n...,"[restructuring, nrule, of, law, ntrue, federal...","[restructuring, nrule, law, ntrue, federalism,...",0.000,0.892,0.108,0.3182,positive
1,2022-11-19 00:26:21+00:00,b'RT @Capt_Aliyu: Buhari will go down as the w...,2,buhari will go down as the worse president in...,"[buhari, will, go, down, as, the, worse, presi...","[buhari, go, worse, president, history, nigeri...",0.214,0.786,0.000,-0.7162,negative
2,2022-11-19 00:18:57+00:00,b'RT @Dec_Uche: Atiku Abubakar is 360 degrees ...,3,atiku abubakar is degrees ahead of other pre...,"[atiku, abubakar, is, degrees, ahead, of, othe...","[atiku, abubakar, degrees, ahead, presidential...",0.000,1.000,0.000,0.0000,neutral
3,2022-11-19 00:16:53+00:00,b'RT @Tony_Ogbuagu: If there is anything you w...,4,if there is anything you will miss don xe x x...,"[if, there, is, anything, you, will, miss, don...","[anything, miss, xe, x, xt, miss, townhall, se...",0.122,0.878,0.000,-0.2960,negative
4,2022-11-19 00:16:24+00:00,b'RT @cousinray_: BREAKING NEWS!! \nMFN Coalit...,5,breaking news nmfn coalition condemns atiku a...,"[breaking, news, nmfn, coalition, condemns, at...","[breaking, news, nmfn, coalition, condemns, at...",0.370,0.630,0.000,-0.8555,negative


In [43]:
atiku_df.comp_score.value_counts()

positive    3959
neutral     3168
negative    2873
Name: comp_score, dtype: int64